In [48]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import altair as alt

Raw data

In [91]:
ipc_sdn= pd.read_csv('sudan/ipc_sdn_national_long.csv').iloc[1:]#
idp_sdn = pd.read_csv('sudan/event_data_sdn.csv').iloc[1:]
acled = pd.read_csv('sudan/ACLED_2020-01-01-2024-12-31-Sudan.csv')

In [ ]:
#remove  the data for the year 2020
acled = acled[acled['year'] != 2020]


In [92]:
# Convert event_date to datetime format
acled['event_date'] = pd.to_datetime(acled['event_date'])
# make a new column for month year
acled['month_year'] = acled['event_date'].dt.to_period('M')

In [94]:
acled.event_type.value_counts()


event_type
Battles                       4742
Protests                      3684
Violence against civilians    3530
Explosions/Remote violence    2908
Strategic developments        2899
Riots                          293
Name: count, dtype: int64

In [102]:
# Group by month year and event_type and count the number of events and sum the fatalities
acled_monthly = acled.groupby(['month_year', 'event_type']).agg({'fatalities':'sum', 'event_date':'count'}).reset_index()
#remove riots
acled_monthly = acled_monthly[acled_monthly['event_type'] != 'Riots']

In [103]:
# Convert month_year to string format for Altair
acled_monthly['month_year'] = acled_monthly['month_year'].astype(str)

# Plot line charts per event_type and show number of events
line = alt.Chart(acled_monthly).mark_line().encode(
    x=alt.X('month_year:T', title='Month-Year'),
    y=alt.Y('event_date:Q', title='Number of Events'),
    color='event_type:N',
    tooltip=['month_year', 'event_type', 'event_date']
).properties(
    title='Number of events per month by event type'
)
line

alt.Chart(...)

In [104]:
#save acled_monthly to csv
acled_monthly.to_csv('sudan/acled_monthly.csv', index=False)

Shapefile

#### APIs

Fews

In [50]:
fews = requests.get('https://fdw.fews.net/api/ipcphase/?format=json&country_code=SD&preference=best&fields=simple')
fews_sudan = fews.json()

fews_sudan = pd.json_normalize(fews_sudan)
fews_sudan = fews_sudan.drop(columns=['source_document', 'data_usage_policy', 'status', 'id', 'collection_schedule', 'collection_status'])

#fews_sudan.head()

ACLED

In [51]:
acled=requests.get('https://api.acleddata.com/acled/read.json?key=bmoYzVNA-Xl6fWpbI!i8&email=c.abdelgelil@lse.ac.uk&country=Sudan&event_date=2021-01-01|2024-12-30&event_date_where=BETWEEN&limit=3000').json()
acled_sud=pd.DataFrame(acled['data'])
#acled_sud

In [64]:
acled_sud.columns

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp'],
      dtype='object')

In [65]:
acled_sud.disorder_type.value_counts()

disorder_type
Political violence                    2278
Strategic developments                 694
Demonstrations                          27
Political violence; Demonstrations       1
Name: count, dtype: int64

IPC

In [52]:
ipc = requests.get('https://api.ipcinfo.org/analyses?country=SD&type=A&periods=C&key=3bdbeef9-fc4c-4315-a665-cc56deb11be6').json()
#ipc

In [53]:
for item in ipc:
    response = requests.get(f"https://api.ipcinfo.org/areas/{item['id']}/C?format=geojson&key=3bdbeef9-fc4c-4315-a665-cc56deb11be6")
    if response.status_code == 200:
        try:
            ipc_geoson = response.json()
            with open(f"ipc_geoson_{item['id']}.json", 'w') as f:
                json.dump(ipc_geoson, f)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for item {item['id']}: {e}")
        except Exception as e:
            print(f"Error saving file for item {item['id']}: {e}")
    else:
        print(f"Error fetching data for item {item['id']}: HTTP {response.status_code}")

Error fetching data for item 19191589: HTTP 404
Error fetching data for item 64768802: HTTP 404
Error fetching data for item 74795267: HTTP 404


HDX HAPI

In [54]:
idp = requests.get('https://hapi.humdata.org/api/v1/affected-people/idps?location_code=SDN&admin_level=0&output_format=json&app_identifier=Y2FtZWxpYWdlbGlsOmNhbWVsaWEuYWJkZWxnZWxpbEBvdXRsb29rLmNvbQ==&limit=10000&offset=0').json()
idp_sdn = pd.json_normalize(idp['data']) #until sep 2023

In [27]:
idp_sdn.columns

Index(['location_ref', 'location_code', 'location_name', 'admin1_ref',
       'admin1_code', 'admin1_name', 'provider_admin1_name', 'admin2_ref',
       'admin2_code', 'admin2_name', 'provider_admin2_name', 'resource_hdx_id',
       'reporting_round', 'assessment_type', 'operation', 'population',
       'reference_period_start', 'reference_period_end'],
      dtype='object')

In [ ]:
idp_sdn = idp_sdn.drop(columns=['location_ref', 'location_code', 'admin1_ref', 'admin1_code', 'admin1_name', 'provider_admin1_name', 'provider_admin1_name', 'provider_admin2_name', ])
idp_sdn

In [40]:
idp_sdn.operation.value_counts()

operation
Darfur conflict                      73
Armed Clashes in Sudan               45
Armed Clashes in Sudan (Overview)    17
Armed Clashes in Sudan (Monthly)      8
Name: count, dtype: int64

In [44]:
#filter by Armed Clashes in Sudan (Monthly)      8
idp_sdn_test = idp_sdn[idp_sdn.operation == 'Armed Clashes in Sudan']
idp_sdn_test

,location_name,admin2_ref,admin2_code,admin2_name,resource_hdx_id,reporting_round,assessment_type,operation,population,reference_period_start,reference_period_end
5,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,2,BA,Armed Clashes in Sudan,334053,2023-04-28T00:00:00,2023-04-28T00:00:00
11,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,3,BA,Armed Clashes in Sudan,736223,2023-05-07T00:00:00,2023-05-07T00:00:00
16,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,4,BA,Armed Clashes in Sudan,843130,2023-05-14T00:00:00,2023-05-14T00:00:00
20,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,5,BA,Armed Clashes in Sudan,1042114,2023-05-21T00:00:00,2023-05-21T00:00:00
24,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,6,BA,Armed Clashes in Sudan,1210214,2023-05-29T00:00:00,2023-05-29T00:00:00
27,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,7,BA,Armed Clashes in Sudan,1428551,2023-06-04T00:00:00,2023-06-04T00:00:00
31,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,8,BA,Armed Clashes in Sudan,1670991,2023-06-13T00:00:00,2023-06-13T00:00:00
35,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,9,BA,Armed Clashes in Sudan,1965946,2023-06-18T00:00:00,2023-06-18T00:00:00
38,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,10,BA,Armed Clashes in Sudan,2152936,2023-06-25T00:00:00,2023-06-25T00:00:00
41,Sudan,33410,None,None,80911e9b-7527-469a-a545-4074860e1288,11,BA,Armed Clashes in Sudan,2231523,2023-07-01T00:00:00,2023-07-01T00:00:00


In [ ]:
foodprices = pd.read_csv('yemen/hdx_hapi_food_price_yem.csv').iloc[1:]#turn into API, monthly, only until 2020
wfp_foodprices = pd.read_csv('yemen/wfp_food_prices_yem.csv').iloc[1:]#turn into API, monthly adm1 +2, only until 2020

poverty = pd.read_csv('yemen/hdx_hapi_poverty_rate_yem.csv').iloc[1:] #adm1 yearly



/var/folders/w9/4s00gf1j7sl8k190_53qgb7c0000gn/T/ipykernel_16042/2885461550.py:1: DtypeWarning: Columns (1,2,10,11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  foodprices = pd.read_csv('yemen/hdx_hapi_food_price_yem.csv').iloc[1:]#turn into API, monthly, only until 2020


ACLED data

In [29]:
acled_yem = acled_yem.drop(columns=['event_id_cnty', 'source', 'source_scale', 'notes', 'tags', 'timestamp'])

In [30]:
acled_yem_grouped = acled_yem.groupby(['disorder_type', 'admin1']).size().reset_index(name='count')


In [31]:
# Extract unique values from both columns
acled_admin1_unique = acled_yem_grouped['admin1'].unique()
adm1_en_unique = geofile_adm1['ADM1REF_EN'].unique()

# Find common and different values
common_admin1 = set(acled_admin1_unique).intersection(set(adm1_en_unique))
different_admin1 = set(acled_admin1_unique).difference(set(adm1_en_unique))

print("Common admin1 values:", common_admin1)
print("Different admin1 values:", different_admin1)

Common admin1 values: {'Hajjah', 'Amran', 'Ibb', 'Sanaa', 'Sadah', 'Raymah', 'Al Jawf', 'Aden', 'Shabwah', 'Dhamar', 'Ad Dali', 'Hadramawt', 'Al Bayda', 'Marib', 'Abyan', 'Al Mahwit'}
Different admin1 values: {'', 'Al Mahrah', 'Suqutra', 'Al Hudaydah', 'Amanat al Asimah', 'Taizz', 'Lahij'}


In [32]:
acled_yem_grouped['admin1'] = acled_yem_grouped['admin1'].replace({ "Taizz": "Taiz",
                                                          "Al Hudaydah": "Al Hodeidah", 
                                                          "Lahij": "",
                                                          "Amanat al Asimah": "Sadah",
                                                        "Suqutra": "Socotra",
                                                        "Al Mahrah": "Al Maharah"})

In [33]:

merged_df = pd.merge(acled_yem_grouped, hunger_map_latest, left_on='admin1', right_on='adminone', how='inner')

# Create Altair scatter plot
chart = alt.Chart(merged_df).mark_circle(size=60).encode(
    x='count:Q',
    y='prevalence:Q',
    color='disorder_type:N',
    tooltip=['adminone', 'count', 'prevalence'],
    facet=alt.Facet('disorder_type:N', columns=1)
).properties(
    title='Prevalence vs Count by Disorder Type'
)

chart.show()


alt.Chart(...)